In [22]:
from typing import List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.integrate import odeint

In [23]:
CSV_PATH = "./covid-19_smitte_external/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv"

POPULATION = 5_831_000 # Statistics Denmark 2023-01-01
INFECTIOUS_DAYS = 14 # mean days an individual is infectious
INFECTED_LIMIT  = 0 # 2**14 # 2**16 # There need to be at least INFECTED_LIMIT infected for us to care

In [24]:
print("Reading", CSV_PATH)
raw = pd.read_csv(
    CSV_PATH,
    sep=";",
    encoding="cp1252", # danish chars
)

Reading ./covid-19_smitte_external/03_bekraeftede_tilfaelde_doede_indlagte_pr_dag_pr_koen.csv


In [25]:
cols_lower: List[str] = [c.lower() for c in raw.columns]

def find_col(substrs: List[str]) -> str:
    for col, col_lc in zip(raw.columns, cols_lower):
        if all(s in col_lc for s in substrs):
            return col
    raise KeyError(f"Column containing {substrs} not found")

col_date      = find_col(["dato"])
col_confirmed = find_col(["bekr", "tilf", "alt"]) # new daily cases
col_deaths    = find_col(["døde"]) # new daily deaths

raw[col_date] = pd.to_datetime(raw[col_date], errors="coerce")
raw = raw.rename(columns={
    col_date: "Date",
    col_confirmed: "Confirmed",
    col_deaths: "Deaths",
})

In [26]:
daily = (
    raw.groupby("Date")[["Confirmed", "Deaths"]]
       .sum(min_count=1)
       .sort_index()
)

# Reconstruct S, I, R
cum = daily.cumsum()
cum.columns = ["Confirmed_cum", "Deaths_cum"]

new_cases = daily["Confirmed"].fillna(0)
I = new_cases.rolling(INFECTIOUS_DAYS, min_periods=1).sum()
R = cum["Confirmed_cum"] - I
S = POPULATION - I - R

series = pd.concat({"S": S, "I": I, "R": R}, axis=1).astype(float).clip(lower=0)

# Trim to interval where I >= INFECTED_LIMIT
valid = series["I"] >= INFECTED_LIMIT
first = valid.idxmax()
last = valid[::-1].idxmax()
series = series.loc[first:last]

In [27]:
# Daily increments
delta_I = series["I"].diff()
delta_R = series["R"].diff()

# Valid rows (exclude zeros/negatives and missing first‐difference)
valid = (series["I"] > 0) & (series["S"] > 0)
gamma_t = (delta_R / series["I"]).where(valid).dropna()
gamma_hat = gamma_t.mean()

beta_t = (
    (delta_I + gamma_hat * series["I"]) /
    (series["S"] * series["I"] / POPULATION)
).where(valid).dropna()
beta_hat = beta_t.mean()

R0_hat = beta_hat / gamma_hat
print(f"Estimated beta = {beta_hat}, gamma = {gamma_hat}  =>  R0 = {R0_hat}")

# analytic (delta-method) 95 % CIs
from scipy.stats import norm
z = norm.ppf(0.975)

n = len(gamma_t)
se_gamma = gamma_t.std(ddof=1) / np.sqrt(n)
se_beta  = beta_t .std(ddof=1) / np.sqrt(n)

gamma_ci = (gamma_hat - z * se_gamma, gamma_hat + z * se_gamma)
beta_ci  = (beta_hat  - z * se_beta , beta_hat  + z * se_beta)

se_R0 = R0_hat * np.sqrt((se_beta / beta_hat) ** 2 + (se_gamma / gamma_hat) ** 2)
R0_ci = (R0_hat - z * se_R0, R0_hat + z * se_R0)

print(f"gamma analytic 95 % CI: {gamma_ci[0]} – {gamma_ci[1]}")
print(f"beta analytic 95 % CI: {beta_ci [0]} – {beta_ci [1]}")
print(f"R0 analytic 95 % CI: {R0_ci[0]} – {R0_ci[1]}")

# block bootstrap 95 % CIs (weekly blocks)
block_len = 7 # one-week blocks
values = pd.DataFrame({"gamma": gamma_t, "beta": beta_t}).reset_index(drop=True)
T = len(values)

def resample_blocks():
    idx = []
    while len(idx) < T:
        start = np.random.randint(0, T - block_len + 1)
        idx.extend(range(start, start + block_len))
    return idx[:T]

B = 2000
boot_gamma = np.empty(B)
boot_beta  = np.empty(B)

for b in range(B):
    sample = values.iloc[resample_blocks()]
    boot_gamma[b] = sample["gamma"].mean()
    boot_beta[b]  = sample["beta" ].mean()

gamma_ci_boot = np.percentile(boot_gamma, [2.5, 97.5])
beta_ci_boot  = np.percentile(boot_beta , [2.5, 97.5])
R0_ci_boot    = np.percentile(boot_beta / boot_gamma, [2.5, 97.5])

print(f"gamma bootstrap 95 % CI: {gamma_ci_boot[0]} – {gamma_ci_boot[1]}")
print(f"beta bootstrap 95 % CI: {beta_ci_boot [0]} – {beta_ci_boot [1]}")
print(f"R0 bootstrap 95 % CI: {R0_ci_boot[0]} – {R0_ci_boot[1]}")


Estimated beta = 0.1321038107327189, gamma = 0.07338095487997708  =>  R0 = 1.8002465482874916
gamma analytic 95 % CI: 0.07200725051500535 – 0.0747546592449488
beta analytic 95 % CI: 0.1281079297326687 – 0.1360996917327691
R0 analytic 95 % CI: 1.7362076326272329 – 1.8642854639477504
gamma bootstrap 95 % CI: 0.07104080362300828 – 0.07594081829256263
beta bootstrap 95 % CI: 0.12292183915135885 – 0.14062519977552876
R0 bootstrap 95 % CI: 1.627581940193657 – 1.9667058209533907
